## Overview
This is a 14-class classification problem.

### Preprocess

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import os
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../../CHESTXRAY/Data_Entry_2017.csv')
df = df.sample(frac=1)  # shuffle dataframe
image_dir = '../../images'
image_path = {f:os.path.join(image_dir,f) for f in os.listdir(image_dir)}
print("Scans found: {}, total headers: {}".format(len(image_path), df.shape[0]))
df['path'] = df['Image Index'].map(image_path.get)  # add path column
# note: df['Patient Age'] has value larger than 100
df.sample(3)

In [ ]:
label_counts = df['Finding Labels'].value_counts()[:15]
fig, ax1 = plt.subplots(1,1,figsize = (12, 6))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
_ = ax1.set_xticklabels(label_counts.index, rotation = 90)

#### one-hot encoding

In [ ]:
from itertools import chain
all_labels = np.unique(list(chain(*df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
print(len(all_labels), all_labels)
for c_label in all_labels:
    df[c_label] = df['Finding Labels'].map(lambda finding: 1 if c_label in finding else 0)
df.to_csv("Data_Entry_2017_shuffled.csv")  # save shuffled data, for convenience of later prediction
df.sample(3)

In [ ]:
classes = ['Atelectasis','Cardiomegaly','Consolidation','Edema','Effusion',
           'Emphysema','Fibrosis','Hernia','Infiltration','Mass','Nodule',
           'Pleural_Thickening','Pneumonia','Pneumothorax']
nb_records, nb_classes = df.shape[0], len(classes)
print(nb_records, nb_classes)

In [ ]:
train_df = df.iloc[:int(nb_records*0.7)]
valid_df = df.iloc[int(nb_records*0.7):int(nb_records*0.9)]
test_df = df.iloc[int(nb_records*0.9):]
print(train_df.shape, valid_df.shape, test_df.shape)

### Build and train model

In [ ]:
from models import ModelFactory

image_shape = (224, 224, 3)  # input image shape
model = ModelFactory(nb_classes, image_shape).densenet121()
model.summary()

In [ ]:
from keras.optimizers import Adam

optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
from class_weights import get_class_weights
class_weights = get_class_weights(df, classes)
print(class_weights)

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

weight_path="weights_{epoch:03d}_{val_loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(weight_path, monitor='acc', verbose=1, 
                             save_best_only=False, mode='max', save_weights_only=True)
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
reduceLR = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1, 
                             mode="min", cooldown=5, min_lr=1e-8)
callbacks = [checkpoint, early, reduceLR]

In [ ]:
from generator import DataGenerator

batch_size = 32

train_generator = DataGenerator(train_df, path_key="path", classes_key=classes, batch_size=batch_size)
valid_generator = DataGenerator(valid_df, path_key="path", classes_key=classes, batch_size=batch_size, shuffle=False)
print(len(train_generator), len(valid_generator))

In [ ]:
nb_epoch = 100

history = model.fit_generator(generator=train_generator, 
                              epochs=nb_epoch, 
                              validation_data=valid_generator, 
                              callbacks=callbacks, 
                              class_weight=class_weights, 
                              verbose=1,
                              workers=6, 
                              shuffle=False)

In [ ]:
import pickle

# dump history
with open("history.pkl", "wb") as f:
    pickle.dump({
        "history": history.history,
    }, f)

### model predict

In [ ]:
model.load_weights("weights_20181018-1019/weights_034_0.3282.hdf5")

In [ ]:
test_generator = DataGenerator(test_df, path_key="path", classes_key=classes, batch_size=batch_size, shuffle=False)
print(len(test_generator))

In [ ]:
y_pred = model.predict_generator(test_generator, verbose=1)
print(type(y_pred), len(y_pred))

In [ ]:
y_test = test_df[classes].values
print(type(y_test), len(y_test))

In [ ]:
print(y_pred[0])
print(y_test[0])

In [ ]:
# binary classification

tp, tn, fp, fn = 0, 0, 0, 0
thres = 0.5  # threshold to determine if was a valid prediction
for r_test, r_pred in zip(y_test, y_pred):
    tp += 1 if r_test.sum() > 0 and r_pred.max() > thres else 0  # predicted desease
    tn += 1 if r_test.sum() == 0 and r_pred.max() <= thres else 0
    fp += 1 if r_test.sum() == 0 and r_pred.max() > thres else 0
    fn += 1 if r_test.sum() > 0 and r_pred.max() <= thres else 0
print("tp = {}, tn = {}, fp = {}, fn = {}\n".format(tp, tn, fp, fn))

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
accuracy = (tp + tn) / (tp + tn + fp + fn)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
print("Accuracy: ", accuracy)